## 1. Create a CSV containing an aggregate table showing the total orders and revenue each store had each month. It should have the following columns:
    
Year (Eg: 2020)
Month (Eg: January)
Store Name
Number of Orders
Total Revenue

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import datetime
import calendar


In [3]:
order = pd.read_csv('orders_test (2).csv')
store = pd.read_csv('store_test (2).csv')

In [4]:
order.duplicated().sum(), store.duplicated().sum()

(0, 0)

In [5]:
order.head(2)

,id,total,customer_id,store_id,order_date
0,1,19.36,21,1,2020-03-03
1,2,8.85,88,8,2020-04-02


In [6]:
order.describe()

,id,total,customer_id,store_id
count,1000.000000,1000.000000,1000.000000,1000.00000
mean,500.500000,15.193990,50.902000,5.38900
std,288.819436,8.420347,28.909517,2.91585
min,1.000000,0.220000,1.000000,1.00000
25%,250.750000,7.860000,26.000000,3.00000
50%,500.500000,15.680000,50.000000,5.00000
75%,750.250000,22.045000,77.000000,8.00000
max,1000.000000,29.970000,100.000000,10.00000


In [7]:
store.head(2)

,id,name,address
0,1,Valdez Inc,"18321 Joseph Lodge\nChristineland, NH 69026"
1,2,Stevens-Barr,Unit 0902 Box 4445\nDPO AE 19637


In [8]:
store.describe()

,id
count,10.00000
mean,5.50000
std,3.02765
min,1.00000
25%,3.25000
50%,5.50000
75%,7.75000
max,10.00000


In [9]:
store =store.rename(columns={"id": "store_id"})

In [10]:
order_1 = pd.merge(order,store,on='store_id', how='left')

In [11]:
order_1.head()

,id,total,customer_id,store_id,order_date,name,address
0,1,19.36,21,1,2020-03-03,Valdez Inc,"18321 Joseph Lodge\nChristineland, NH 69026"
1,2,8.85,88,8,2020-04-02,"Martinez, Marsh and Gomez","1434 Shelby Streets Apt. 134\nWest Austin, SD ..."
2,3,5.53,41,3,2020-03-03,Taylor Ltd,"3433 Hill Forest\nCharlesmouth, KY 16037"
3,4,12.90,96,8,2020-03-15,"Martinez, Marsh and Gomez","1434 Shelby Streets Apt. 134\nWest Austin, SD ..."
4,5,8.19,25,7,2020-01-21,Hall Inc,23446 Brittney Isle Apt. 423\nSouth Victoriabe...


In [12]:
order_1.index = pd.to_datetime(order_1['order_date'])

In [13]:
total_orders = order_1.groupby(by=[order_1.index.month, order_1.index.year,order_1.name])['total'].agg(['sum','count'])

In [14]:
total_orders_df = pd.DataFrame(total_orders)
total_orders_df.index.names = ['Month','Year','Store name']
total_orders_df.reset_index(inplace=True)

In [15]:
total_orders_df = total_orders_df[['Year','Month','Store name','count','sum']]
total_orders_df = total_orders_df.rename(columns={"Store name": "Store Name", "count": "Number of Orders", "sum":"Total Revenue"})

In [16]:
total_orders_df['Month'] = total_orders_df['Month'].apply(lambda x: calendar.month_name[x])

In [17]:
total_orders_df.head()

,Year,Month,Store Name,Number of Orders,Total Revenue
0,2020,January,"Adams, Barrett and Thomas",27,315.79
1,2020,January,Edwards-Mcdaniel,18,280.72
2,2020,January,Hall Inc,21,348.17
3,2020,January,"Henderson, Olson and Ward",23,282.65
4,2020,January,"Martinez, Marsh and Gomez",17,222.38


In [35]:
total_orders_df.to_csv('Store Revenue.xlsx',index=False)

## 2. Create a CSV containing a list of users who have placed less than 10 orders. It should have the following columns:
    
First Name
Last Name
Email
Orders Placed by user

In [19]:
import hashlib

In [20]:
customer = pd.read_csv('customer_test (2).csv')

In [21]:
customer.head(2)

,id,first_name,last_name,email
0,1,Sara,Ramirez,samantha67@yahoo.com
1,2,Joshua,Jimenez,richardtimothy@hotmail.com


In [22]:
customer.duplicated().sum()

0

In [23]:
customer = customer.rename(columns={"id": "customer_id"})

In [24]:
order_cus = pd.merge(order,customer,on='customer_id', how='left')

In [25]:
order_cus.head()

,id,total,customer_id,store_id,order_date,first_name,last_name,email
0,1,19.36,21,1,2020-03-03,Alexis,Ferguson,hayesdavid@yahoo.com
1,2,8.85,88,8,2020-04-02,Christopher,Diaz,christinemcguire@yahoo.com
2,3,5.53,41,3,2020-03-03,Rebecca,Little,cynthiaoliver@gmail.com
3,4,12.90,96,8,2020-03-15,Leah,Sandoval,michael53@dominguez-blackwell.com
4,5,8.19,25,7,2020-01-21,Richard,Peterson,nancy53@hotmail.com


In [26]:
order_placed = order_cus.groupby(['email','first_name','last_name'])['store_id'].count()

In [27]:
order_placed_df = pd.DataFrame(order_placed)
order_placed_df.index.names = ['Email','First Name','Last Name']
order_placed_df.reset_index(inplace=True)

In [28]:
order_placed_df = order_placed_df.rename(columns={"store_id": "No. of orders Placed by user"})
order_placed_df = order_placed_df[['First Name','Last Name','Email','No. of orders Placed by user']]

In [29]:
order_placed_df = order_placed_df[order_placed_df['No. of orders Placed by user'] < 10]

In [30]:
order_placed_df.reset_index(drop=True, inplace=True)

In [31]:
order_placed_df[order_placed_df['No. of orders Placed by user'] >= 10]

,First Name,Last Name,Email,No. of orders Placed by user


In [32]:
for i in range(len(order_placed_df.Email)):
    res = hashlib.md5(order_placed_df.Email[i].encode())
    order_placed_df['Email'][i] = res.hexdigest()

In [33]:
order_placed_df.head()

,First Name,Last Name,Email,No. of orders Placed by user
0,Thomas,Montgomery,2a011e8e05b91ce7c5fda8fb679506f1,8
1,Travis,Phelps,a750ecf95c82cc5ff2d2124339ed8884,7
2,Shelby,Osborn,8503336e0f3a2be920984c4f9c6db980,5
3,Joanna,Terry,f9477c84df3fa1addf8fb94a5bd41b6c,7
4,James,Washington,71a32f6bd1ba1407a89a7d42323391a0,8


In [36]:
order_placed_df.to_csv('Number of orders placed.xlsx',index=False)